In [7]:
import numpy as np
import pandas as pd
import sklearn.metrics as mt
import matplotlib.pyplot as plt
import os

# from scipy.stats import kendalltau
# from scipy.stats import spearmanr

In [8]:
label = pd.read_csv("/mnt/yzy/NIMCGCN/datasets/data(MDA108)/m-d.csv",delimiter=',',header=None).values

In [9]:
def AU_ROC(label, pred):
    fpr, tpr, thresholds = mt.roc_curve(label, pred, pos_label=1)
    roc_auc = mt.auc(fpr, tpr)

    best_idx = np.argmax(tpr - fpr)
    best_threshold = thresholds[best_idx]
    # print(best_threshold)
    return roc_auc, best_threshold

def AU_PRC(label, pred):
    precision, recall, thresholds = mt.precision_recall_curve(label, pred)
    prc_auc = mt.auc(recall, precision)
    return prc_auc

def MRR(label, pred):
    mrr = mt.label_ranking_average_precision_score([label],[pred])
    return mrr

def HIT(label, pred, K):
    sorted_indices = np.argsort(pred)[::-1]
    hits = label[sorted_indices[:K]].sum()
    return hits/K

def MSE(label, pred):
    mse = mt.mean_squared_error(label, pred)
    return mse

# def CI(label, pred):
#     CI_k, pv_k = kendalltau(label, pred)
#     CI_s, pv_s = spearmanr(label, pred)    
#     return pv_k, pv_s

In [22]:
# 单个或者多个结果分析
count = 0
label_1d = label.reshape(-1)
'''key: name of the model'''
'''vakue: prediction score'''
scores = {"label":label_1d}

scores["MultiHead"] = np.load("/mnt/yzy/NIMCGCN/Prediction/Compare/MultiHead_MultiViewSAGE_10FoldCV_230519_[lr]0.0005_[wd]0.0_[ep]150_[cvMthd]elem_[miRDim]64_[drugDim]64_[kFdim]64_[alpha]0.3_[loss]WMSE.npy")
scores["Channel"] = np.load("/mnt/yzy/NIMCGCN/Prediction/Compare/ChannelAtt_MultiViewSAGE_10FoldCV_230519_[lr]0.0005_[wd]0.0_[ep]150_[cvMthd]elem_[miRDim]64_[drugDim]64_[kFdim]64_[alpha]0.3_[loss]WMSE.npy")
scores["NoAtt"] = np.load("/mnt/yzy/NIMCGCN/Prediction/Compare/noAtt_MultiViewSAGE_10FoldCV_230519_[lr]0.0005_[wd]0.0_[ep]150_[cvMthd]elem_[miRDim]64_[drugDim]64_[kFdim]64_[alpha]0.3_[loss]WMSE.npy")

for name,pred in scores.items():
    count += 1
    pred_1d = pred.reshape(-1)
    
    if pred_1d.shape[0] != label_1d.shape[0]:
        print("Error: {} and {} have different shape".format(dir, name))
        continue

    '''基于打分的指标'''
    '''计算AUC'''
    auc,threshold = AU_ROC(label_1d, pred_1d)
    auc = 1 - auc if auc < 0.5 else auc

    '''计算AU_PRC'''
    aupr = AU_PRC(label_1d, pred_1d)

    '''计算MRR'''
    mrr = MRR(label_1d, pred_1d)

    '''计算HIT'''
    hit_50  = HIT(label_1d, pred_1d, 50)
    hit_100 = HIT(label_1d, pred_1d, 100)

    '''计算MSE'''
    mse = MSE(label_1d, pred_1d)

    '''基于标签的指标'''
    pred_y = (pred_1d >= threshold).astype(int)
    '''分类指标'''
    acc = mt.accuracy_score(label_1d, pred_y)
    recall = mt.recall_score(label_1d, pred_y)
    precision = mt.precision_score(label_1d, pred_y,zero_division=0)
    f1 = mt.f1_score(label_1d, pred_y)


    exp_method = name.split("_")[0]
    # print("{}/{}: AUC = {:.8f}, AUPR = {:.8f}, MRR = {:.8f}, HIT = {:.8f}, ACC = ".format(dir, exp_method, auc, aupr, mrr, hit))
    print("{:10}/{:15}:\t AUC =  {:.5f}, AUPR = {:.5f}, MRR = {:.5f}, HIT@50 = {:.5f}, HIT@100 = {:.5f}, MSE = {:.5f}, ACC = {:.5f}, RECALL = {:.5f}, PRECISION = {:.5f}, F1 = {:.5f}"\
            .format(dir, exp_method, auc, aupr, mrr, hit_50, hit_100, mse, acc, recall, precision, f1))

print(count)

MF        /label          :	 AUC =  1.00000, AUPR = 1.00000, MRR = 1.00000, HIT@50 = 1.00000, HIT@100 = 1.00000, MSE = 0.00000, ACC = 1.00000, RECALL = 1.00000, PRECISION = 1.00000, F1 = 1.00000
MF        /MultiHead      :	 AUC =  0.50550, AUPR = 0.00063, MRR = 0.00064, HIT@50 = 0.00000, HIT@100 = 0.00000, MSE = 0.00185, ACC = 0.79432, RECALL = 0.22448, PRECISION = 0.00069, F1 = 0.00138
MF        /Channel        :	 AUC =  0.54655, AUPR = 0.00073, MRR = 0.00073, HIT@50 = 0.00000, HIT@100 = 0.00000, MSE = 0.00763, ACC = 0.45898, RECALL = 0.61818, PRECISION = 0.00072, F1 = 0.00144
MF        /NoAtt          :	 AUC =  0.50523, AUPR = 0.00062, MRR = 0.00062, HIT@50 = 0.00000, HIT@100 = 0.00000, MSE = 0.25090, ACC = 0.22915, RECALL = 0.79790, PRECISION = 0.00065, F1 = 0.00131
4


In [15]:
# 所有结果联合分析 并写入文件
output = open("./metric_output_5f_@50@100.txt", "w+")

count = 0
label_1d = label.reshape(-1)
for dir in os.listdir("Prediction/"):
    for file in os.listdir("Prediction/{}".format(dir)):
        count += 1

        pred = np.load("Prediction/{}/{}".format(dir,file))
        pred_1d = pred.reshape(-1)
        
        if pred_1d.shape[0] != label_1d.shape[0]:
            print("Error: {} and {} have different shape".format(dir, file))
            continue

        '''基于打分的指标'''
        '''计算AUC'''
        auc,threshold = AU_ROC(label_1d, pred_1d)
        auc = 1 - auc if auc < 0.5 else auc

        '''计算AU_PRC'''
        aupr = AU_PRC(label_1d, pred_1d)

        '''计算MRR'''
        mrr = MRR(label_1d, pred_1d)

        '''计算HIT'''
        hit_50  = HIT(label_1d, pred_1d, 50)
        hit_100 = HIT(label_1d, pred_1d, 100)

        '''计算MSE'''
        mse = MSE(label_1d, pred_1d)

        '''基于标签的指标'''
        pred_y = (pred_1d >= threshold).astype(int)
        '''分类指标'''
        acc = mt.accuracy_score(label_1d, pred_y)
        recall = mt.recall_score(label_1d, pred_y)
        precision = mt.precision_score(label_1d, pred_y,zero_division=0)
        f1 = mt.f1_score(label_1d, pred_y)


        exp_method = file.split("_")[0]
        # print("{}/{}: AUC = {:.8f}, AUPR = {:.8f}, MRR = {:.8f}, HIT = {:.8f}, ACC = ".format(dir, exp_method, auc, aupr, mrr, hit))
        print("{:10}/{:15}:\t AUC =  {:.5f}, AUPR = {:.5f}, MRR = {:.5f}, HIT@50 = {:.5f}, HIT@100 = {:.5f}, MSE = {:.5f}, ACC = {:.5f}, RECALL = {:.5f}, PRECISION = {:.5f}, F1 = {:.5f}"\
              .format(dir, exp_method, auc, aupr, mrr, hit_50, hit_100, mse, acc, recall, precision, f1))
        output.write("{:10}/{:15}:\t AUC =  {:.5f}, AUPR = {:.5f}, MRR = {:.5f}, HIT@50 = {:.5f}, HIT@100 = {:.5f}, MSE = {:.5f}, ACC = {:.5f}, RECALL = {:.5f}, PRECISION = {:.5f}, F1 = {:.5f}\n"\
              .format(dir, exp_method, auc, aupr, mrr, hit_50, hit_100, mse, acc, recall, precision, f1))

output.close()
print(count)

Nimc      /GCN            :	 AUC =  0.50001, AUPR = 0.00032, MRR = 0.00063, HIT@50 = 0.00000, HIT@100 = 0.00000, MSE = 0.00063, ACC = 0.99937, RECALL = 0.00000, PRECISION = 0.00000, F1 = 0.00000
Nimc      /SAGE           :	 AUC =  0.95772, AUPR = 0.45290, MRR = 0.45296, HIT@50 = 1.00000, HIT@100 = 1.00000, MSE = 0.00045, ACC = 0.96603, RECALL = 0.87832, PRECISION = 0.01614, F1 = 0.03169
Nimc      /SAGE           :	 AUC =  0.75636, AUPR = 0.01215, MRR = 0.01231, HIT@50 = 0.16000, HIT@100 = 0.11000, MSE = 0.00063, ACC = 0.83968, RECALL = 0.59091, PRECISION = 0.00233, F1 = 0.00464
Nimc      /GAT            :	 AUC =  0.82124, AUPR = 0.01757, MRR = 0.01773, HIT@50 = 0.16000, HIT@100 = 0.11000, MSE = 0.00063, ACC = 0.79885, RECALL = 0.73217, PRECISION = 0.00230, F1 = 0.00459
Nimc      /SAGE           :	 AUC =  0.52165, AUPR = 0.19462, MRR = 0.04362, HIT@50 = 1.00000, HIT@100 = 0.61000, MSE = 0.00061, ACC = 0.99937, RECALL = 0.04336, PRECISION = 0.50820, F1 = 0.07990
Nimc      /SAGE          